In [7]:
h1 = ["S" , "A"]        
h2 = ["A", "H", "S"]
h3 = ["A"]
h4 = ["F", "S"]

hashtags = [h1, h2, h3, h4]

In [14]:
def map_to_dict(hashtags):
    h_dict = {}
    for t in hashtags:
        #check length > 1
        if len(t) > 1:
            print(t)
            if len(t) == 2:
                new_t = t
            else: 
                new_t = t + [t[0]]
            print(dict(zip(new_t[:-1], new_t[1:])))
            
            
            

In [15]:
map_to_dict(hashtags)

['S', 'A']
{'S': 'A'}
['A', 'H', 'S']
{'A': 'H', 'S': 'A', 'H': 'S'}
['F', 'S']
{'F': 'S'}


### above is not great, because it doesn't work for a tweet containing four hashtags

## Using sets

In [25]:
hashtags = set(["A", "B", "C", "D"])


In [60]:

def build_edges(hashtag_set):
    #set of sets: inner sets are tuples defining an edge
    #input hashtags should be a set
    edges = set([])
    for h in hashtags:
        for remaining_h in hashtags.difference(h):
            #frozenset creates an immutable set
            edge = frozenset([h, remaining_h])
            edges.add(edge)
    return edges
            
        
            

In [61]:
build_edges(hashtags)

{frozenset({'C', 'D'}),
 frozenset({'B', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'D'}),
 frozenset({'B', 'D'}),
 frozenset({'A', 'B'})}

In [62]:
def process_tweets(hashtags):
    graph = set([])
    for hashtag_set in hashtags:
        edges = build_edges(hashtag_set)
        #add edges to graph
        for edge in edges:
            graph.add(edge)
    return graph
    
        

In [63]:
hashtags = [set(["A", "B", "C", "D"]), set(["A", "B", "E"])]

In [65]:
result = process_tweets(hashtags)
print(result)

{frozenset({'C', 'D'}), frozenset({'A', 'C'}), frozenset({'C', 'B'}), frozenset({'A', 'D'}), frozenset({'B', 'D'}), frozenset({'E', 'A'}), frozenset({'A', 'B'}), frozenset({'E', 'B'})}


In [67]:
from collections import defaultdict

h_count = defaultdict(int)
for edge in result:
    for h in edge:
        h_count[h] += 1

print(h_count)
        
        

defaultdict(<class 'int'>, {'E': 2, 'C': 3, 'D': 3, 'B': 4, 'A': 4})


## Test Stream Storage

In [6]:
import json
import pandas as pd
path =  "../../tweet_input/test_tweets.txt"

In [7]:
with open(path, 'r') as f:
    for tweet in f:
        tweet_dict = json.loads(tweet)
        

In [17]:
#pandas frame
data = {"edge": set(["A", "B"]), "time": "Fri Oct 30 15:29:45 +0000 2015"}
df = pd.DataFrame(data)

In [18]:
df

,edge,time
0,"{A, B}",Fri Oct 30 15:29:45 +0000 2015
1,"{A, B}",Fri Oct 30 15:29:45 +0000 2015


### add edge

In [19]:
more_data = {"edge": set(["A", "C"]), "time": "Fri Oct 30 15:22:45 +0000 2015"}

In [21]:
df.append(more_data, ignore_index=True)

,edge,time
0,"{A, B}",Fri Oct 30 15:29:45 +0000 2015
1,"{A, B}",Fri Oct 30 15:29:45 +0000 2015
2,"{A, C}",Fri Oct 30 15:22:45 +0000 2015


### Filter by time

In [25]:
data = {"edge": set(["A", "B"]), "time": pd.Timestamp("Fri Oct 30 15:29:45 +0000 2015")}

df2 = pd.DataFrame(data)


In [28]:
more_data = {"edge": set(["A", "C"]), "time": pd.Timestamp("Fri Oct 30 15:22:45 +0000 2015")}

In [36]:
this_time = pd.Timestamp("Fri Oct 30 15:22:45 +0000 2015")

In [37]:
df2.append([set(["A", "C"]), this_time])

TypeError: 'Timestamp' object is not iterable

In [38]:
pd.Timestamp("Fri Oct 30 15:22:45 +0000 2015")

Timestamp('2015-10-30 15:22:45+0000', tz='UTC')

## Try again


In [41]:
data = {"edge": set(["A", "B"]), "time_str": "Fri Oct 30 15:29:45 +0000 2015"}


In [42]:
df = pd.DataFrame(data)

In [43]:
df

,edge,time_str
0,"{A, B}",Fri Oct 30 15:29:45 +0000 2015
1,"{A, B}",Fri Oct 30 15:29:45 +0000 2015


In [51]:

df2 = pd.DataFrame({"edge": set(["A", "C"]), "time_str": pd.Timestamp("Fri Oct 30 15:21:35 +0000 2015")})

,edge,time_str
0,"{A, C}",2015-10-30 15:21:35+00:00
1,"{A, C}",2015-10-30 15:21:35+00:00


In [52]:
df.append(df2)

TypeError: data type not understood

## One more time, leave to Pandas attempt



In [80]:
data = {"edge": set(["A", "B"]), "time_str": pd.Timestamp("Fri Oct 30 15:29:45 +0000 2015")}
df = pd.DataFrame(data)

In [84]:
df = df.append([{"edge": set(["A", "C"]), "time_created": pd.Timestamp("Fri Oct 30 15:22:45 +0000 2015")}], ignore_index=True)

TypeError: data type not understood

In [64]:
df[df["edge"] == set(["A", "C"])]

,edge,time_str
0,"{A, C}",2015-10-30 15:22:45+00:00
1,"{A, C}",2015-10-30 15:22:45+00:00


In [79]:
df.time_str.dtype

datetime64[ns, UTC]

In [86]:
df.dtypes

edge                     object
time_str    datetime64[ns, UTC]
dtype: object

## try without sets

In [105]:
data = {"edge": set(["A;B"]), "time_str": [pd.Timestamp("Fri Oct 30 15:29:45 +0000 2015")]}

df = pd.DataFrame(data)

In [106]:
df

,edge,time_str
0,{A;B},2015-10-30 15:29:45+00:00


In [109]:
df.append([set(["C;B"]), [pd.Timestamp("Fri Oct 30 15:29:45 +0000 2015")]], ignore_index=True)

TypeError: data type not understood

In [123]:
df = pd.DataFrame({"edge": [("A", "B")], "time": ["2014"]} )

In [124]:
df

,edge,time
0,"(A, B)",2014


In [126]:
df.append([("C", "D"), "2018"], gnore_index=True)

,edge,time,0,1,2,3
0,"(A, B)",2014,NaN,NaN,NaN,NaN
1,NaN,NaN,C,D,None,None
2,NaN,NaN,2,0,1,8


In [128]:
df = pd.DataFrame({"edge": [set(("A", "B")), set(("B", "C"))], 
                  "time": [pd.Timestamp("Fri Oct 30 15:29:45 +0000 2015"),
                           pd.Timestamp("Mon Oct 30 15:22:45 +0000 2015")]
                   })

In [129]:
df

,edge,time
0,"{A, B}",2015-10-30 15:29:45+00:00
1,"{B, C}",2015-10-30 15:22:45+00:00


In [135]:
df2 = pd.DataFrame({"edge": [set(("C", "D")), set(("E", "F"))], 
                  "time": [pd.Timestamp("Fri Oct 29 12:29:45 +0000 2015"),
                           pd.Timestamp("Mon Oct 30 11:22:45 +0000 2015")]
                   })

In [136]:
df2

,edge,time
0,"{D, C}",2015-10-29 12:29:45+00:00
1,"{F, E}",2015-10-30 11:22:45+00:00


In [137]:
df.append(df2)

,edge,time
0,"{A, B}",2015-10-30 15:29:45+00:00
1,"{B, C}",2015-10-30 15:22:45+00:00
0,"{D, C}",2015-10-29 12:29:45+00:00
1,"{F, E}",2015-10-30 11:22:45+00:00


In [139]:
df = df.append(df2)

In [155]:
df3 = pd.DataFrame({"edge": [set(("G", "D")),], 
                  "time": [pd.Timestamp("Fri Jan 29 12:29:45 +0000 2015"),]
                   })

In [156]:
df.append(df3)

,edge,time
0,"{A, B}",2015-10-30 15:29:45+00:00
1,"{B, C}",2015-10-30 15:22:45+00:00
0,"{D, C}",2015-10-29 12:29:45+00:00
1,"{F, E}",2015-10-30 11:22:45+00:00
0,"{D, G}",2015-01-29 12:29:45+00:00


In [158]:
df.sort_values("time").head(1)

,edge,time
0,"{D, C}",2015-10-29 12:29:45+00:00


In [159]:
df.sort_values("time").head(1).time + pd.Timedelta('60 sec')

0   2015-10-29 12:30:45+00:00
Name: time, dtype: datetime64[ns, UTC]